In [6]:
library(caret)
library(kernlab)

groups <- read.csv(file="./MovementAAL/groups/MovementAAL_DatasetGroup.csv",head=TRUE,sep=",")
targetAll <- read.csv(file="./MovementAAL/dataset/MovementAAL_target.csv",head=TRUE,sep=",")

#Group 1
allDataGroup1<-list()
allDataGroup1Target<-list()
groups1 = groups[groups$dataset_ID==1, ]

index<-1
for (id in groups1$X.sequence_ID){
    caminho <-paste("./MovementAAL/dataset/MovementAAL_RSS_",id,".csv",sep="")
    allDataGroup1[[index]]<-read.csv(file=caminho,head=TRUE,sep=",")
    allDataGroup1Target[index]<-targetAll[[2]][id]
    index<-index+1
}

In [7]:
wtData <- NULL
minStepsBack = 1
for (i in 1:length(allDataGroup1)){
     aa<- t(unlist(allDataGroup1[[i]][(nrow(allDataGroup1[[i]])-minStepsBack):nrow(allDataGroup1[[i]]),]))
    wtData <- rbind(wtData, aa)
}
wtData <- as.data.frame(wtData)
data = unlist(allDataGroup1Target)
target = factor(data,labels=c("No","Yes"))
frameDataFinal <- data.frame(cbind(target, wtData))
head(frameDataFinal)
##use only lagged data

,target,X.RSS_anchor11,X.RSS_anchor12,RSS_anchor21,RSS_anchor22,RSS_anchor31,RSS_anchor32,RSS_anchor41,RSS_anchor42
1,Yes,0,-0.14286,0.04,-0.6,-0.047619,-0.28571,-0.05,-0.1
2,Yes,-0.33333,-0.14286,0.04,0.04,0.095238,0.14286,-0.1,0.05
3,Yes,-0.28571,-0.14286,-0.04,-0.08,-0.095238,0.14286,-0.05,0.35
4,Yes,-0.42857,-0.57143,-0.2,-0.52,0,0.047619,0.4,-0.15
5,Yes,-0.57143,-0.52381,-0.6,-0.44,0.57143,0.28571,0.9,0.8
6,Yes,-1,-0.47619,-0.32,-0.2,0.71429,0.52381,0.65,0.95


In [8]:
# set.seed(998)
inTraining <- createDataPartition(frameDataFinal$target, p = .7, list = FALSE)
training <- frameDataFinal[ inTraining,]
testing  <- frameDataFinal[-inTraining,]
fitControl <- trainControl(method = "none", classProbs = TRUE)

In [12]:
By default, the function keeps only the PCs that are necessary to explain at least 95% of 
    the variability in the data, but this can be changed through the argument thresh

ERROR: Error in parse(text = x, srcfile = src): <text>:1:4: unexpected symbol
1: By default
       ^


In [10]:
set.seed(825)
svmLinearFit <- train(target ~ ., data = training, preProcess=c("pca"),
                 method = "svmLinear",
                 trControl = fitControl,
                 family=binomial)

In [11]:
preds<- predict(svmLinearFit, newdata = testing, type = "prob")
head(preds)
preds<- predict(svmLinearFit, newdata = testing)
length(preds)
# predict(gbmFit4, newdata = testing, type = "prob")
table(preds,testing$target)


,No,Yes
1,0.3647666,0.6352334
2,0.439686,0.560314
3,0.1080454,0.8919546
4,0.126103,0.873897
5,0.2381788,0.7618212
6,0.1169478,0.8830522


[1] 30

     
preds No Yes
  No  10   2
  Yes  5  13